In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import bootstrap
from scipy.stats import ttest_ind

In [ ]:
mTypes = {'D1':['D1#14', 'D1#15', 'D1#21', 'D1#7', 'D1#33'],
          'D2':['D2#13', 'D2#24', 'D2#36', 'D2#23', 'D2#38', 'D2#52']}

In [ ]:
col_names = ["dis", "time", "group", "Day", "list", "mice", "itr"]
dtypes={
        "dis" : float, 
        "time" : int, 
        "group" : str, 
        "Day" : str, 
        "list" : str, 
        "mice" : str, 
        "itr" : int
        }
read_df = pd.read_csv("pca_distance_example.csv", dtype=dtypes)
df_grouped = read_df.groupby(['time', 'group', 'list', 'mice', 'itr']).dis.sum().to_frame()
df_grouped

In [ ]:
colNames = ['dis']
times = np.arange(1,302)
iter = np.arange(99)

In [ ]:
my_index = pd.MultiIndex(levels=[[],[],[],[]], codes=[[],[],[],[]], names=[u'itr', u'group', u'time', u'type'])
df_grouped_mType_average = pd.DataFrame([], columns=['dis'], index=my_index, dtype=float)
for mType in mTypes:
    for t in times:
        for it in range(0,100):
            df_grouped_mType_average.loc[it, 'ALP', t, mType] = df_grouped.loc[t, 'ALP', mType, :, it]['dis'].mean()
            df_grouped_mType_average.loc[it, 'BL2', t, mType] = df_grouped.loc[t, 'BL2', mType, :, it]['dis'].mean()
df_grouped_mType_average

In [ ]:
def my_statistic(sample1, sample2, axis):
    statistic, _ = ttest_ind(sample1, sample2, axis=-1)
    return statistic

In [ ]:
FRAC_PART = 5
colTimes = []
addr = -10.25
while addr <= 10: 
    colTimes.append(addr)
    addr += 1*FRAC_PART/10
    addr = np.round(addr, 2)
colNames = ["[{t1},{t2})".format(t1 = colTimes[i], t2 = colTimes[i+1]) for i in range(len(colTimes) - 1)]
print(colNames)
print(len(colNames))

In [ ]:
for rat in ['D1', 'D2']:
    counter = 0
    colVals = []
    lower, higher = [], []
    for time in range(48, 247, 5):
        slice = np.arange(time, time+5)
        colVals.append(colNames[counter])
        print(colNames[counter], slice)
        counter += 1
        ALP_dat = df_grouped.loc[slice, 'ALP', rat, :, :]['dis']
        BL2_dat = df_grouped.loc[slice, 'BL2', rat, :, :]['dis']
        bootstrap_val = bootstrap((ALP_dat, BL2_dat), my_statistic, method='basic', confidence_level=0.95)
        lower.append(bootstrap_val.confidence_interval[0])
        higher.append(bootstrap_val.confidence_interval[1])
    plt.fill_between(colVals, lower, higher, color='r', alpha=.15)
    plt.xticks(rotation=90)
    plt.axvline('[-5.25,-4.75)', alpha=0.5)
    plt.axvline('[-2.75,-2.25)', alpha=0.5)
    plt.ylabel('Z-score')
    plt.title(rat)
    plt.show()
    plt.clf()


In [ ]:
time_slices = [[98, 99, 100, 101, 102], [123, 124, 125, 126, 127], [148, 149, 150, 151, 152]] # -5, -2.5, 0
for rat in ['D1', 'D2']:
    for slice in time_slices:
        ALP_dat = df_grouped_mType_average.loc[:, 'ALP', slice, rat]['dis']
        BL2_dat = df_grouped_mType_average.loc[:, 'BL2', slice, rat]['dis']
        bootstrap_val = bootstrap((ALP_dat, BL2_dat), my_statistic, method='basic', confidence_level=0.95)
        print(bootstrap_val)

In [ ]:
idx = ['Type', 'Day', 'Rodent', 'iter']
mind_df = pd.read_csv('mind_svm_output_ALP_example.csv', index_col=idx)
mind_df_shuff = pd.read_csv('mind_svm_output_control_example.csv', index_col=idx)
mind_df

In [ ]:
index = pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=[u'Type', u'iter'], dtype=(str, int))
mind_df_grouped = pd.DataFrame([], columns=colNames, index=index, dtype=float)
mind_df_grouped_shuff = pd.DataFrame([], columns=colNames, index=index, dtype=float)
mind_df_grouped

In [ ]:
iter = np.arange(100)
for mType in mTypes:
    for it in iter:
        mind_df_grouped.loc[(mType, it),:] = mind_df.loc[mType, 'Day11', :, it].mean()
        mind_df_grouped_shuff.loc[(mType, it),:] = mind_df_shuff.loc[mType, 'Day11', :, it].mean()
mind_df_grouped

In [ ]:
for rat in ['D1', 'D2']:
    lower, higher = [], []
    for tSlice in colNames:
        ALP_dat = mind_df_grouped.loc[rat, :][tSlice]
        BL2_dat = mind_df_grouped_shuff.loc[rat, :][tSlice]
        bootstrap_val = bootstrap((ALP_dat, BL2_dat), my_statistic, method='basic', confidence_level=0.95)
        if tSlice in ['[-5.0,-4.5)', '[-2.5,-2.0)']:
            print(rat, tSlice, bootstrap_val)
        lower.append(bootstrap_val.confidence_interval[0])
        higher.append(bootstrap_val.confidence_interval[1])
    plt.fill_between(colNames, lower, higher, color='r', alpha=.15)
    plt.xticks(rotation=90)
    plt.axvline('[-5.0,-4.5)', alpha=0.5)
    plt.axvline('[-2.5,-2.0)', alpha=0.5)
    plt.ylabel('Z-score')
    plt.title(rat)
    plt.show()
    plt.clf()